# BPE Tutorial

> This NB contains the code for training a BPE

In [ ]:
import pandas as pd

from icodegen.data.core import *
from pathlib import Path
from sklearn.model_selection import train_test_split

In [ ]:
path = Path('../dvc-icodegen')
df = pd.read_csv(path/'searchnet/[codesearchnet-java-1597073966.81902].csv', sep = '~', index_col = 0)
df.head()

In [ ]:
len(df)

In [ ]:
df_trn = df[df.partition == 'train'].copy()
df_val = df[df.partition == 'valid'].copy()
df_tst = df[df.partition == 'test'].copy()
print(len(df_trn))
df_trn.head()

In [ ]:
assert ((pd.concat([df_tst, df_trn, df_val]) == df).all()).all()

In [ ]:
df_new_trn, df_bpe = train_test_split(df_trn, test_size = 0.1) # 10% selected to match the Maybe Deep Neural Networks are the Best Choice for Modeling Source Code paper

In [ ]:
len(df_new_trn), len(df_bpe)

In [ ]:
df_bpe.partition = ['bpe'] * len(df_bpe)

In [ ]:
df_bpe.head()

In [ ]:
df_new = pd.concat([df_new_trn, df_bpe, df_val, df_tst], ignore_index = True)
df_new.head()

In [ ]:
len(df_new[df_new.partition == 'train']), len(df_new[df_new.partition == 'bpe']), len(df_new[df_new.partition == 'valid']), len(df_new[df_new.partition == 'test'])

In [ ]:
assert len(df_new) == len(df)

In [ ]:
df_non_ascii = remove_non_ascii(df_new)
df_replaced = replace_special_tokens(df_non_ascii, java_special_tokens)
len(df_replaced[df_replaced.partition == 'train']), len(df_replaced[df_replaced.partition == 'bpe']), len(df_replaced[df_replaced.partition == 'valid']), len(df_replaced[df_replaced.partition == 'test'])

In [ ]:
df_replaced.to_csv(path/'searchnet/[codesearchnet-java-1597073966.81902].csv', quoting = 1, sep = '~')
df_replaced_new = pd.read_csv(path/'searchnet/[codesearchnet-java-1597073966.81902].csv', index_col = 0, quoting = 1, sep = '~')

assert (df_replaced == df_replaced_new).all().all()

In [28]:
tokenizer = train_tokenizer(df_replaced[df_replaced.partition == 'bpe'], java_special_tokens)
tokenizer.save(str(path/'bpe/tokenizer-java.json'))
tokenizer.get_vocab_size()

10000

In [33]:
from tokenizers import Tokenizer

new_tokenizer = Tokenizer.from_file(str(path/'bpe/tokenizer-java.json'))
new_tokenizer.encode(' <&&> <{><').tokens

['Ġ', '<&&>', 'Ġ', '<{>', 'Ġ<']

In [ ]:
new_tokenizer.get_vocab()